In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/
!git clone https://{token}@github.com/kytomic/fake-image-classification.git
# Please Enter Your GitHub Token for Cloning the Project
token = ''


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/fake-image-classification/

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
# Using GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Data Preparation
## real vs fake images (casia dataset)
https://www.kaggle.com/code/shaft49/real-vs-fake-images-casia-dataset

Please download the dataset from the link on the discord forum as there is requirements for the file structure.

In [ ]:
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "real_and_fake_face"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")

In [ ]:
train_dir = image_path / "training"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

random.seed(35)
image_path_list = list(image_path.glob("*/*/*.jpg"))
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem
img = Image.open(random_image_path)


print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
print(f"Image name: {str(random_image_path).replace('data/real_and_fake_face/training/fake/', '')}")
img

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.ToTensor()
])

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=32,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=32,
                             num_workers=1,
                             shuffle=False)

# Error Level Analysis

In [ ]:
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]

In [ ]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'

    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)

    ela_im = ImageChops.difference(im, resaved_im)

    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff

    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)

    return ela_im

In [ ]:
img

In [ ]:
convert_to_ela_image('data/real_and_fake_face2/training/fake/easy_91_1000.jpg', 90)

In [ ]:
train_real_path_list = list(image_path.glob("training/real/*.jpg"))
train_fake_path_list = list(image_path.glob("training/fake/*.jpg"))
test_real_path_list = list(image_path.glob("test/real/*.jpg"))
test_fake_path_list = list(image_path.glob("test/fake/*.jpg"))

In [ ]:
def convert_all_to_ela(train_real_path_list, train_fake_path_list, test_real_path_list, test_fake_path_list):
  for img_name in train_real_path_list:
    img = convert_to_ela_image(str(img_name), 90)
    name = str(img_name).replace('data/real_and_fake_face/training/real/', '')
    img.save("data/preprocessed_images/training/real/" + name, "JPEG")

  for img_name in train_fake_path_list:
    img = convert_to_ela_image(str(img_name), 90)
    name = str(img_name).replace('data/real_and_fake_face/training/fake/', '')
    img.save("data/preprocessed_images/training/fake/" + name, "JPEG")

  for img_name in test_real_path_list:
    img = convert_to_ela_image(str(img_name), 90)
    name = str(img_name).replace('data/real_and_fake_face/test/real/', '')
    img.save("data/preprocessed_images/test/real/" + name, "JPEG")

  for img_name in test_fake_path_list:
    img = convert_to_ela_image(str(img_name), 90)
    name = str(img_name).replace('data/real_and_fake_face/test/fake/', '')
    img.save("data/preprocessed_images/test/fake/" + name, "JPEG")

#convert_all_to_ela(train_real_path_list, train_fake_path_list, test_real_path_list, test_fake_path_list)

In [ ]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "preprocessed_images"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")

train_dir = image_path / "training"
test_dir = image_path / "test"

data_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

class_names = train_data.classes

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=4,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=4,
                             num_workers=1,
                             shuffle=False)

# Model Creation


In [ ]:
class FakeImageModel(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*32*32, out_features=output_shape)
    )

  # def xavier_initialization(self, m):
  #   if isinstance(m, nn.Conv2d):
  #     nn.init.xavier_uniform(m.weight)
  #     m.bias.data.fill_(0.01)

  # def init_weights(self):
  #   self.block_1.apply(self.xavier_initialization)
  #   self.block_2.apply(self.xavier_initialization)


  def forward(self, x):
    out = self.block_1(x)
    # print('Block 1: ', out.shape)

    out = self.block_2(out)
    # print('Block 2: ', out.shape)

    out = self.classifier(out)
    return out

torch.manual_seed(42)
model = FakeImageModel(input_shape=3, hidden_units=16, output_shape=len(class_names))
# model.init_weights()
model

In [ ]:
def accuracy(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100

    return acc

In [ ]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "real_and_fake_face"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")

train_dir = image_path / "training"
test_dir = image_path / "test"

data_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

class_names = train_data.classes

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=4,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=4,
                             num_workers=1,
                             shuffle=False)

In [ ]:
# Setup loss function and optimization algorithm
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

In [ ]:
# Training Loop
from tqdm.auto import tqdm
torch.manual_seed(42)
epochs = 10
train_losses, test_losses = [], []
train_accs, test_accs = [], []
i = 0

for epoch in tqdm(range(epochs)):
  print(f'Epoch: {epoch} -----')
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(train_dataloader):
    model.train()
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    train_acc += accuracy(y_true=y, y_pred=y_pred.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 200 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%\n")

  model.eval()
  with torch.inference_mode():
    test_loss, test_acc = 0.0, 0.0

    for X, y in test_dataloader:
      test_pred = model(X)
      test_loss += loss_fn(test_pred, y).item()
      test_acc += accuracy(y_true=y, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
print('Training accuracies: ', train_accs)
print('Test accuracies: ', test_accs)

# DenseNet + ELA

In [ ]:
from torchvision import models

model = models.densenet121(weights='DenseNet121_Weights.IMAGENET1K_V1')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# Create model save path
MODEL_NAME = "dense-fake-image-model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Note: loading model will error if the shapes here aren't the same as the saved version
loaded_model = models.densenet121(weights='DenseNet121_Weights.IMAGENET1K_V1')

# Load in the saved state_dict()
loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# Send model to GPU
loaded_model = loaded_model.to(device)

# Training and Evaluation

In [ ]:
# Setup loss function and optimization algorithm
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), 0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

In [ ]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "preprocessed_images"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")

train_dir = image_path / "training"
test_dir = image_path / "test"

data_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

class_names = train_data.classes

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=4,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=4,
                             num_workers=1,
                             shuffle=False)

In [ ]:
# Training Loop
from tqdm.auto import tqdm
torch.manual_seed(42)
epochs = 10
train_losses, test_losses = [], []
train_accs, test_accs = [], []
i = 0

for epoch in tqdm(range(epochs)):
  print(f'Epoch: {epoch} -----')
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(train_dataloader):
    model.train()
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    train_acc += accuracy(y_true=y, y_pred=y_pred.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 200 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%\n")

  model.eval()
  with torch.inference_mode():
    test_loss, test_acc = 0.0, 0.0

    for X, y in test_dataloader:
      test_pred = model(X)
      test_loss += loss_fn(test_pred, y).item()
      test_acc += accuracy(y_true=y, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
    test_losses.append(train_loss)
    test_accs.append(train_acc)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
model.eval()
with torch.inference_mode():
  test_loss, test_acc = 0.0, 0.0
  for X, y in test_dataloader:
    test_pred = model(X)
    test_loss += loss_fn(test_pred, y)
    test_acc += accuracy(y_true=y, y_pred=test_pred.argmax(dim=1))

  test_loss /= len(test_dataloader)
  test_acc /= len(test_dataloader)
  print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
print('Training accuracies: ', train_accs)
print('Test accuracies: ', test_accs)

In [ ]:
from pathlib import Path

# Create models directory (if it doesn't already exist)
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# Create model save path
MODEL_NAME = "dense-fake-image-model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
!git status

In [ ]:
!git add main.ipynb models

In [ ]:
!git commit -m 'CNN and ELA Code adjustment'

In [ ]:
!git config --global user.email "kytomic1@gmail.com"

In [ ]:
!git push --set-upstream origin build_cnn